# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
geoapify_key

'd27d93a63ede46bf8ae470c0f919f152'

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,isafjordur,66.0755,-23.1240,6.54,96,100,17.10,IS,1695099917
1,1,whitehorse,60.7161,-135.0538,8.90,60,75,6.17,CA,1695099426
2,2,edinburgh of the seven seas,-37.0676,-12.3116,13.12,81,94,10.13,SH,1695099918
3,3,robertsport,6.7533,-11.3686,25.19,86,100,2.65,LR,1695099918
4,4,waitangi,-43.9535,-176.5597,12.12,81,2,3.13,NZ,1695099918


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,brak,27.5496,14.2714,22.48,55,0,4.20,LY,1695099920
18,18,galliano,29.4422,-90.2992,22.22,88,0,0.00,US,1695099921
55,55,la manga del mar menor,37.6413,-0.7165,24.00,88,0,2.06,ES,1695099931
71,71,palmas,-10.2128,-48.3603,24.93,53,0,2.57,BR,1695099713
171,171,montepuez,-13.1256,38.9997,22.53,65,0,1.93,MZ,1695099959
320,320,la marina de port,41.3603,2.1376,23.95,76,0,2.57,ES,1695099994
322,322,longboat key,27.4125,-82.6590,26.68,59,0,3.09,US,1695099696
352,352,boa vista,2.8197,-60.6733,26.99,69,0,2.06,BR,1695100002
398,398,maymana,35.9214,64.7836,22.13,20,0,1.55,AF,1695100013
412,412,tamanrasset,22.7850,5.5228,21.95,15,0,2.57,DZ,1695100016


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
11,brak,LY,27.5496,14.2714,55,
18,galliano,US,29.4422,-90.2992,88,
55,la manga del mar menor,ES,37.6413,-0.7165,88,
71,palmas,BR,-10.2128,-48.3603,53,
171,montepuez,MZ,-13.1256,38.9997,65,
320,la marina de port,ES,41.3603,2.1376,76,
322,longboat key,US,27.4125,-82.6590,59,
352,boa vista,BR,2.8197,-60.6733,69,
398,maymana,AF,35.9214,64.7836,20,
412,tamanrasset,DZ,22.7850,5.5228,15,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accomodation.hotel",
    "apiKey":geoapify_key,
    "limit":20   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
brak - nearest hotel: No hotel found
galliano - nearest hotel: No hotel found
la manga del mar menor - nearest hotel: No hotel found
palmas - nearest hotel: No hotel found
montepuez - nearest hotel: No hotel found
la marina de port - nearest hotel: No hotel found
longboat key - nearest hotel: No hotel found
boa vista - nearest hotel: No hotel found
maymana - nearest hotel: No hotel found
tamanrasset - nearest hotel: No hotel found
sarakhs - nearest hotel: No hotel found
de queen - nearest hotel: No hotel found
al bab - nearest hotel: No hotel found
agua prieta - nearest hotel: No hotel found
zahedan - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
11,brak,LY,27.5496,14.2714,55,No hotel found
18,galliano,US,29.4422,-90.2992,88,No hotel found
55,la manga del mar menor,ES,37.6413,-0.7165,88,No hotel found
71,palmas,BR,-10.2128,-48.3603,53,No hotel found
171,montepuez,MZ,-13.1256,38.9997,65,No hotel found
320,la marina de port,ES,41.3603,2.1376,76,No hotel found
322,longboat key,US,27.4125,-82.6590,59,No hotel found
352,boa vista,BR,2.8197,-60.6733,69,No hotel found
398,maymana,AF,35.9214,64.7836,20,No hotel found
412,tamanrasset,DZ,22.7850,5.5228,15,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel", "Country"]
)

# Display the map
# YOUR CODE HERE
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country)